In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os
import torchvision
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import *
from torchvision.transforms import *

print("Testing if GPU is available.")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    Warning("GPU is not available, Falling back to CPU.")

os.chdir("/home/aaron/git/ImageColorization/")
BASEADDR = os.getcwd()
print(f"Current working directory: {BASEADDR}")

BATCH_SIZE = 64

Testing if GPU is available.
GPU is available and being used.
Current working directory: /home/aaron/git/ImageColorization


: 

# Download and unpack imagenet64 database
imagenet64 is the imagenet database with each image scaled to be 64 by 64 pixels.

In [56]:
checkpoints = f"{BASEADDR}/checkpoints/"
content = f"{BASEADDR}/content/"
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

import shutil
if not os.path.exists(content):
    os.makedirs(content)
if not os.path.exists(f'{content}/imagenet64'):
  os.chdir(content)
  if not os.path.exists(content + 'imagenet64.tar'):
    print("Downloading archive...")
    !wget https://pjreddie.com/media/files/imagenet64.tar
  print("Uncompressing...")
  !tar -xf imagenet64.tar

print("Data ready!")

Data ready!


In [57]:
class ColorizationDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load color image
        color_img, _ = self.dataset[idx]

        # Convert color image to grayscale
        gray = transforms.Grayscale()
        grayscale_img = gray(color_img)

        return grayscale_img, color_img

In [58]:
def get_imagenet64_data(batchSize):
  transform=transforms.Compose([
                             transforms.RandomResizedCrop(64, scale=(1.0, 1.0), ratio=(1., 1.)),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor()
                          ])

  # load training dataset
  trainSet = torchvision.datasets.ImageFolder(root=f'{content}/imagenet64/train/', transform=transform)
  colorizationTrainSet = ColorizationDataset(trainSet)
  trainLoader = DataLoader(colorizationTrainSet, batch_size=batchSize, shuffle=True, num_workers=0)

  # load testing dataset
  testSet = torchvision.datasets.ImageFolder(root=f'{content}/imagenet64/val/', transform=transform)
  colorizationTestSet = ColorizationDataset(testSet)
  testloader = torch.utils.data.DataLoader(colorizationTestSet, batch_size=batchSize, shuffle=False, num_workers=0)
  
  return {'train': trainLoader, 'test': testloader}

In [59]:
class GrayNet(nn.Module):
    def __init__(self, num_input_channels, num_output_channels, initial_channels=16):
        super(GrayNet, self).__init__()

        # Initial number of channels
        in_channels = num_input_channels

        # Define the initial layers of the network
        self.layers = nn.ModuleList([
            nn.Conv2d(num_input_channels, initial_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        ])

        # Update the number of input channels
        in_channels = initial_channels

        # Add more layers based on your desired architecture
        for _ in range(3):  # You can customize the number of layers
            self.layers.extend([
                nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
            ])

        # Final output layer
        self.output_layer = nn.Conv2d(in_channels, num_output_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Forward pass through all layers
        for layer in self.layers:
            x = layer(x)

        # Final output
        x = self.output_layer(x)
        return x
    
testInput = torch.randn(1,1,64,64).to(device)
testNetwork = GrayNet(1,3).to(device)
print(testNetwork)
testNetwork(testInput)

GrayNet(
  (layers): ModuleList(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
  )
  (output_layer): Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


tensor([[[[ 0.0392,  0.0582,  0.0598,  ...,  0.0602,  0.0527,  0.0504],
          [ 0.0452,  0.0593,  0.0416,  ...,  0.0509,  0.0637,  0.0382],
          [ 0.0475,  0.0503,  0.0594,  ...,  0.0703,  0.0614,  0.0305],
          ...,
          [ 0.0395,  0.0643,  0.0732,  ...,  0.0751,  0.0734,  0.0509],
          [ 0.0392,  0.0547,  0.0733,  ...,  0.0626,  0.0678,  0.0472],
          [ 0.0170,  0.0249,  0.0297,  ...,  0.0311,  0.0274,  0.0123]],

         [[ 0.0546,  0.0807,  0.0931,  ...,  0.1022,  0.0794,  0.0819],
          [ 0.0512,  0.0805,  0.0731,  ...,  0.0640,  0.0749,  0.0650],
          [ 0.0624,  0.0781,  0.0784,  ...,  0.0694,  0.0844,  0.0555],
          ...,
          [ 0.0644,  0.0803,  0.0764,  ...,  0.0646,  0.0648,  0.0706],
          [ 0.0593,  0.0743,  0.0758,  ...,  0.0849,  0.0788,  0.0729],
          [ 0.0580,  0.0479,  0.0462,  ...,  0.0579,  0.0438,  0.0446]],

         [[-0.0552, -0.0425, -0.0590,  ..., -0.0475, -0.0463, -0.0575],
          [-0.0528, -0.0509, -

In [60]:
def mutate_architecture(parent_architecture):
    # Copy the parent architecture
    child_architecture = parent_architecture.copy()

    # Randomly select an operation (e.g., add a layer, remove a layer, change a layer's parameters)
    operation = np.random.choice(["add_layer", "remove_layer", "modify_layer"])

    if operation == "add_layer":
        # Add a new layer to the architecture
        child_architecture.append(nn.Conv2d(16, 16, kernel_size=3, padding=1))  # Define the new layer

    elif operation == "remove_layer":
        # Remove a random layer from the architecture
        if len(child_architecture) > 1:  # Ensure there's at least one layer
            index_to_remove = np.random.choice(len(child_architecture))
            del child_architecture[index_to_remove]

    elif operation == "modify_layer":
        # Modify the parameters of a random layer in the architecture
        if len(child_architecture) > 0:
            index_to_modify = np.random.choice(len(child_architecture))
            # Modify the parameters of the layer, e.g., change kernel size, channels, etc.

    return child_architecture

In [61]:
def evaluate(architecture, data_loader, num_epochs=5):
    # Assuming you have access to labeled training data and a loss function
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(architecture.parameters(), lr=0.001)

    architecture.to(device)
    for epoch in range(num_epochs):
        for inputs, targets in tqdm(data_loader, leave=False):
            optimizer.zero_grad()
            outputs = architecture(inputs.to(device))
            loss = criterion(outputs, targets.to(device))
            loss.backward()
            optimizer.step()

    # After training, assess the performance on a validation set or the entire dataset
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, leave=False):
            outputs = architecture(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * len(targets)
            total_samples += len(targets)

    average_loss = total_loss / total_samples
    return -average_loss  # Return the negative loss as a fitness score for maximization

In [62]:

def evolutionary_algorithm(population_size, generations, data_loader):
    population = [GrayNet(1,3) for _ in range(population_size)]

    for generation in range(generations):
        print(f"Generation {generation + 1}/{generations}")

        # Evaluate each individual in the population
        fitness_scores = []
        for individual in population:
            fitness = evaluate(individual.to(device), data_loader)
            fitness_scores.append(fitness)

        # Select the top performers
        top_indices = np.argsort(fitness_scores)[-int(0.2 * population_size):]

        # Create a new population through crossover and mutation
        new_population = [population[i] for i in top_indices]

        while len(new_population) < population_size:
            # Randomly select parents from the top performers
            parent1 = population[np.random.choice(top_indices)]
            parent2 = population[np.random.choice(top_indices)]

            # Crossover and mutation
            child_architecture = mutate_architecture(parent1.architecture + parent2.architecture)
            child = GrayNet(1,3)  # Initialize the child with the mutated architecture
            new_population.append(child)

        population = new_population

    # Return the best individual
    best_index = np.argmax(fitness_scores)
    return population[best_index]




In [63]:
data = get_imagenet64_data(BATCH_SIZE)
trainLoader = data['train']
testLoader = data['test']
best_individual = evolutionary_algorithm(population_size=10, generations=5, data_loader=testLoader)

Generation 1/5


  0%|          | 0/782 [00:00<?, ?it/s]

KeyboardInterrupt: 